In [17]:
# Step 1: Read trusted profiles and create a set of trusted usernames
trusted_profiles_df = pd.read_csv('./dissimilarity/trusted_profiles_info.csv', index_col=0)
trusted_usernames = set(trusted_profiles_df['username'])
len(trusted_usernames)

123

In [12]:
import pandas as pd
import networkx as nx

# Step 1: Read trusted profiles and create a set of trusted usernames
trusted_profiles_df = pd.read_csv('./dissimilarity/trusted_profiles_info.csv', index_col=0)
trusted_usernames = set(trusted_profiles_df['username'])

# Step 2: Read user friends data and build a graph
user_friends_df = pd.read_csv('./normal/user_friends.csv')
G = nx.from_pandas_edgelist(user_friends_df, 'username', 'friend_username')

# Step 3: Initialize trust scores
initial_trust_scores = {username: 1 if username in trusted_usernames else 0 for username in G.nodes}

# Step 4: Propagate trust scores using PageRank or a similar algorithm
# Here we use a damping factor of 0.85 for the PageRank algorithm
trust_scores = nx.pagerank(G, personalization=initial_trust_scores, alpha=0.9)

# Step 5: Normalize trust scores (assuming you want them in range [0, 1])
max_score = max(trust_scores.values())
normalized_trust_scores = {username: score / max_score for username, score in trust_scores.items()}

# Step 6: Map trust scores back to profile indices
# This assumes that there's a way to map usernames to profile indices, which could be another CSV file or part of the existing CSV files.
# Let's assume we have a DataFrame that maps usernames to indices:
username_to_index_df = pd.read_csv('./dissimilarity/all_profile.csv', index_col=0)
username_to_index = pd.Series(username_to_index_df.index.values, index=username_to_index_df.username).to_dict()

# Map the normalized trust scores to profile indices
index_trust_scores = {username_to_index[username]: score for username, score in normalized_trust_scores.items() if username in username_to_index}

# Convert the trust scores to a DataFrame for further analysis
trust_scores_df = pd.DataFrame(list(index_trust_scores.items()), columns=['profile_index', 'trust_score'])


In [13]:
trust_scores_df.to_csv('trust_scores.csv')

In [ ]:
from IPython.display import FileLink

# Replace 'path_to_output.csv' with your actual file path
output_file_path = '/mnt/data/trust_scores.csv'
display(FileLink(output_file_path))
